In [1]:
logB = 32
B = 2**logB

In [2]:
class FFT:
    def generator(self,n):
        return e**(-2*i*pi / (3*n));
    
    def __init__(self,n):
        self.n = n;

        fac = factor(n);
    
        if len(fac) == 2:
            self.Radix2 = fac[0][1];
            self.Radix3 = fac[1][1];

        if len(fac) == 1:
            if fac[0][0] == 2:
                self.Radix2 = fac[0][1];
                self.Radix3 = 0;

            if fac[0][0] == 3:
                self.Radix2 = 0;
                self.Radix3 = fac[0][1];

        self.level = self.Radix2+self.Radix3;

        self.w = self.generator(n);
        self.omega = self.w**n;
    
        tree = zero_matrix(ZZ,self.level+1,n);
        tree[0,0] = 3*n;

        x = self.w**0;
        x1 = round(x.real() * B)
        x2 = round(x.imag() * B)
        self.zetas = [(x1,x2)];
                
        #Radix-2 NTT with Cyclotomic Polynomial
        tree[1,0] = tree[0,0] / 6;
        tree[1,1] = 5*tree[0,0] / 6;

        x = self.w**(tree[1,0]);
        x1 = round(x.real() * B)
        x2 = round(x.imag() * B)
        self.zetas.append((x1,x2));
    
        #Radix-3 NTT
        for ll in range(1,self.Radix3+1):
            for ii in range(2*3**(ll-1)):
                tree[ll+1,3*ii  ] = tree[ll  ,  ii] / 3;
                tree[ll+1,3*ii+1] = tree[ll  ,  ii] / 3 + tree[0,0]/3;
                tree[ll+1,3*ii+2] = tree[ll  ,  ii] / 3 + 2*tree[0,0]/3;

                x = self.w**(tree[ll+1,3*ii]);
                x1 = round(x.real() * B)
                x2 = round(x.imag() * B)
                self.zetas.append((x1,x2));
                x = self.w**(tree[ll+1,3*ii]*2);
                x1 = round(x.real() * B)
                x2 = round(x.imag() * B)
                self.zetas.append((x1,x2));

        #Radix-2 NTT
        for ll in range(self.Radix3+1,self.level):
            for ii in range(2*3**(self.Radix3)*2**(ll-(self.Radix3+1))):
                tree[ll+1,2*ii  ] = tree[ll  ,  ii] / 2;
                tree[ll+1,2*ii+1] = tree[ll  ,  ii] / 2 + tree[0,0]/2;

                x = self.w**(tree[ll+1,2*ii]);
                x1 = round(x.real() * B)
                x2 = round(x.imag() * B)
                self.zetas.append((x1,x2));

        x = self.w**n;
        x1 = round(x.real() * B)
        x2 = round(x.imag() * B)
        self.omega = (x1,x2);
        
        print(tree)

    def print_zetas(self):
        
        l = len(self.zetas);
        str1 = "static const complex_fp64_32 fft_zetas[%d] = {" % l
        print(str1)
        
        str = "\t";
        for ii in range(l):
            zeta1 = self.zetas[ii];
            
            str += "{%+11d, %+11d}, " % (zeta1[0], zeta1[1])
            
            if(ii % 2 == 1):
                str += "\n\t";
      
        print(str);
        print("};");
    
        print("====omega====");
        str = "static const complex_fp64_32 fft_omega = ";
        str += "{%+11d, %+11d};" % (self.omega[0], self.omega[1]);
        print(str)

    
    def fft(self, a):
        k = 1;

        b = [0] * self.n;
        for i in range(self.n):
            b[i] = (round(a[i]*B), 0);

        step = self.n/2;
        
        zeta = self.zetas[k];
        k = k+1;

        for ii in range(self.n/2):
            t = complex_mul(b[ii + step], zeta);

            b[ii + step] = (b[ii][0] - t[0] + b[ii+step][0], b[ii][1] - t[1] + b[ii+step][1]);
            b[ii       ] = (b[ii][0] + t[0],                 b[ii][1] + t[1]);

        step = self.n/6
        while step >= self.n/(6 * 3**(self.Radix3-1)):
            for start in range(0,self.n, 3*step):
                zeta1 = self.zetas[k];
                k = k + 1;
                zeta2 = self.zetas[k];
                k = k + 1;
    
                for ii in range(start,start+step):                    
                    t1 = complex_mul(b[ii +   step], zeta1);
                    t2 = complex_mul(b[ii + 2*step], zeta2);
                    t3 = (t1[0] - t2[0], t1[1] - t2[1]);
                    t3 = complex_mul(t3,self.omega);           

                    t = [0] * 6;
                    t[0] = b[ii][0] - t1[0] - t3[0];
                    t[1] = b[ii][1] - t1[1] - t3[1];
                    t[2] = b[ii][0] - t2[0] + t3[0];
                    t[3] = b[ii][1] - t2[1] + t3[1];
                    t[4] = b[ii][0] + t1[0] + t2[0];
                    t[5] = b[ii][1] + t1[1] + t2[1];
                    
                    b[ii + 2*step] = (t[0], t[1]);
                    b[ii +   step] = (t[2], t[3]);
                    b[ii         ] = (t[4], t[5]);
        
            step = step / 3;

        step = 2^(self.Radix2-2)
        while step >= 1:
            for start in range(0,self.n,(step << 1)):
                zeta = self.zetas[k];
                k += 1;

                for ii in range(start,start+step):
                    t = complex_mul(b[ii + step], zeta);
                    
                    b[ii + step] = (b[ii][0] - t[0], b[ii][1] - t[1]);
                    b[ii       ] = (b[ii][0] + t[0], b[ii][1] + t[1]);
            
            step >>= 1;
        
        return b;

In [3]:
def complex_mul(a,b):
    t1 = (a[0]*b[0] + (1 << (logB-1))) >> logB;
    t2 = (a[0]*b[1] + (1 <<(logB-1))) >> logB;
    t3 = (a[1]*b[0] + (1 <<(logB-1))) >> logB;
    t4 = (a[1]*b[1] + (1 <<(logB-1))) >> logB;

    return (t1 - t4, t2 + t3);

In [4]:
n = 768

In [5]:
fft = FFT(n)

[2304    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0

In [6]:
fft.print_zetas()

static const complex_fp64_32 fft_zetas[768] = {
	{+4294967296,          +0}, {+2147483648, -3719550787}, 
	{+4035949075, -1468965330}, {+3290135830, -2760751762}, 
	{ -745813244, -4229717092}, {-4035949075, +1468965330}, 
	{+4229717092,  -745813244}, {+1468965330, -4035949075}, 
	{-2760751762, -3290135830}, {+2760751762, -3290135830}, 
	{-1468965330, -4035949075}, {-4229717092,  -745813244}, 
	{+4278623649,  -374331065}, { -374331065, -4278623649}, 
	{+3518231241, -2463492036}, {-2463492036, -3518231241}, 
	{+1815131613, -3892562305}, {-3892562305, -1815131613}, 
	{+3892562305, -1815131613}, {-1815131613, -3892562305}, 
	{+2463492036, -3518231241}, {-3518231241, -2463492036}, 
	{ +374331065, -4278623649}, {-4278623649,  -374331065}, 
	{+4290879439,  -187343842}, { -187343842, -4290879439}, 
	{+2901637847, -3166582050}, {-3166582050, -2901637847}, 
	{+4096183113, -1291521575}, {-1291521575, -4096183113}, 
	{+1983195193, -3809682520}, {-3809682520, -1983195193}, 
	{+3622338677, -23076842

In [7]:
#768
g = [-1, 0, -1, 0, 0, 0, -1, 0, 0, 1, -1, -1, 0, 1, -1, 0, -1, 1, 1, 1, -1, 1, 1, -1, -1, 0, -1, 0, 0, 0, 1, -1, -1, 1, 0, -1, 0, 0, 0, -1, -1, 0, 1, -1, 0, 1, -1, 1, 0, -1, -1, 0, 0, 0, -1, 1, 0, -1, 1, -1, 0, -1, 0, 1, -1, -1, 0, 1, 0, -1, -1, -1, 0, 0, -1, 0, 0, -1, -1, -1, 1, 0, -1, 0, 0, 1, 0, -1, 1, 0, -1, 0, -1, 0, 1, -1, 1, 0, -1, 0, -1, 0, 1, -1, -1, -1, 1, -1, 0, -1, -1, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, -1, 0, -1, -1, 1, 0, 1, 0, 0, -1, 0, 0, -1, 1, 1, 1, 0, 1, -1, 1, 0, 0, 1, 0, -1, 0, -1, -1, -1, -1, 0, -1, 1, 0, -1, 1, 1, 0, -1, -1, 0, 0, 1, 0, -1, 0, 1, -1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 1, 0, 1, -1, 0, -1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, -1, 1, -1, 1, 1, 0, 1, -1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 1, -1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, -1, -1, -1, -1, -1, 0, 0, -1, 0, 0, 0, -1, 1, -1, 0, 0, 0, 0, -1, 1, 1, 0, -1, 1, 1, 0, 1, -1, 1, 0, 0, 1, 0, 0, 0, -1, 0, -1, 0, -1, 0, 0, 1, 0, -1, 0, 0, 1, -1, 0, 0, -1, 0, -1, 0, 1, 1, 0, 0, 0, -1, 0, 1, 0, -1, 0, -1, 0, 0, -1, 1, -1, -1, 1, 1, 0, -1, 0, 1, -1, 0, 1, 1, 0, 0, -1, -1, 0, 0, 1, -1, 0, -1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, -1, -1, 1, 1, 1, 1, 0, 0, 0, 0, -1, 1, 0, 1, 1, -1, 0, 1, -1, 0, -1, -1, -1, -1, 0, 0, 1, 1, 1, 0, 0, 0, -1, 1, 1, 0, 1, 1, -1, 0, 0, -1, 0, 1, 0, -1, 0, -1, -1, -1, 0, 0, -1, 0, 0, 1, 0, 0, -1, -1, -1, -1, 0, 0, -1, -1, 0, 0, -1, 0, 0, 1, -1, 1, 0, 0, 0, 0, 0, -1, -1, 1, 1, 0, 0, 0, -1, 0, 1, 0, 1, -1, -1, -1, 1, 0, 0, 1, 0, -1, 0, 0, 0, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 0, 0, -1, 1, 0, 0, 1, 0, 0, -1, 0, 0, 0, -1, 1, -1, -1, -1, 0, 0, 0, -1, 0, -1, 0, 0, -1, -1, -1, 0, 1, 1, 0, 0, 1, 0, 0, -1, 1, 1, 0, 1, 1, 1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, -1, 0, 0, 0, 0, -1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, -1, 0, 0, 0, -1, 1, 1, 1, -1, 1, 0, 0, 0, 0, -1, -1, -1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, -1, 0, 0, -1, -1, 0, 0, 0, 0, -1, 0, 0, -1, -1, 0, 1, 1, -1, 0, 0, 1, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, -1, 0, 0, 0, -1, -1, 0, 0, 1, 0, -1, 0, 0, -1, 1, 0, 0, 0, 0, -1, 0, 0, -1, -1, 0, 0, 0, 0, -1, 0, 0, 1, 0, 1, -1, -1, 0, 0, 1, 0, 0, 1, 0, 1, -1, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, -1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0]
f = [1, -1, 1, 1, -1, 0, -1, 1, 0, 0, 0, 0, 1, 1, 0, 0, -1, 0, 1, 0, 0, 1, -1, 1, -1, 0, 0, 0, 1, -1, -1, 0, 0, 0, -1, 1, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 1, 0, -1, -1, -1, -1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, -1, -1, 1, 0, 1, 0, -1, 1, 0, 0, 1, 0, -1, -1, 1, 1, 0, -1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, -1, 0, 1, 0, -1, 1, 0, 1, 0, 0, -1, -1, 0, 0, -1, 1, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, -1, -1, 1, 1, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, -1, 1, 0, 0, 1, -1, 0, 1, 1, -1, 0, 0, 0, 1, 1, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 1, 1, -1, 0, 0, 0, -1, 0, 1, 1, -1, 1, 1, 0, 0, 0, 0, 1, 0, 0, -1, 1, 0, -1, 0, 0, 0, 0, -1, 0, 1, 1, -1, 1, 0, -1, 1, -1, 0, 0, 1, -1, 1, 0, -1, 0, 0, 1, 0, -1, -1, 1, 1, 0, -1, 0, -1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 1, 0, 0, -1, 0, 1, -1, 1, -1, -1, 1, 1, -1, 0, -1, 1, 0, -1, 1, 1, 0, 0, -1, -1, 1, 1, 0, -1, 0, 1, 0, 1, 0, -1, 0, 1, 1, -1, -1, 1, 0, -1, 0, -1, 0, 1, -1, -1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, -1, -1, 0, 0, 1, 0, 1, -1, 0, 0, 1, 0, 0, 0, -1, 0, 0, 1, 1, -1, 0, 1, 1, -1, 0, 0, -1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 1, -1, -1, 0, -1, 0, 1, 0, 0, 0, -1, -1, -1, 1, -1, 0, 0, -1, 0, 0, 1, 1, 0, -1, 1, 1, 0, 1, 1, 0, -1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, -1, 0, -1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, -1, 0, 1, 0, -1, 1, 1, 0, 1, 0, 0, 1, 1, 0, -1, -1, 0, -1, 1, 0, 0, 1, 1, 0, 1, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, -1, -1, 0, 0, 0, 0, -1, -1, -1, 0, 1, 0, -1, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, -1, 0, 1, -1, 0, -1, 0, 1, 0, 1, 0, 1, -1, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, -1, 0, 1, 1, 1, 0, 0, 0, 0, -1, -1, 1, -1, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, -1, 1, 1, 0, 1, 0, 0, 0, -1, 0, 0, 1, 1, 0, 0, 1, 0, -1, -1, 1, 1, 0, 1, 0, -1, -1, 0, 1, 1, 1, 0, -1, 0, 1, 0, 0, 0, 0, 0, 1, -1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, -1, 1, -1, 0, -1, 0, 0, 1, 1, -1, 0, 1, 0, 0, 0, 0, 1, -1, -1, 0, 1, 0, 0, -1, 0, -1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, -1, -1, -1, 0, 1, -1, 1, 1, -1, 0, 0, 0, 0, -1, 1, 0, 1, 0, 0, 1, -1, 0, 0, 1, -1, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, -1, 0, 1, -1, 0, 0, 0, 0, 0, 1, -1, 0, 1, 0, 0, 0, 1, -1, -1, 0, 0, -1, 0, 1, 1, -1, -1, 0, 1, 0, -1, -1, -1, 0, 0, 0, -1, 1, 1, -1, -1, 0, -1, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 1, -1, -1, 0, 0, 0, 1, 0, 1, 0, 1, -1, -1, 0, 0, 1, 0, 0, -1, -1, -1, 1, 1, 0, 1, 1, 0, -1, -1, 1, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 1, -1, -1, 0, -1, 1, 0, 0, 0, 1, 1]

In [8]:
ghat = fft.fft(g)

In [9]:
for i in range(n):
    print((ghat[i][0] + (1 << logB-1)) >> logB)

-24
-11
-13
16
-4
-4
-18
-18
-12
-18
-3
-11
-11
30
-12
23
-2
-2
10
-1
-8
8
-3
-4
14
24
-14
-1
-24
1
-10
17
1
6
3
18
-4
17
-3
18
-2
-6
6
-3
-1
-18
-15
-13
-7
1
3
-17
-5
-2
-19
13
-1
-15
10
10
13
-22
-22
2
12
21
2
-5
6
-29
21
-15
1
-7
1
22
-2
-6
-8
-7
4
-1
3
19
8
17
6
5
-16
10
-17
-3
22
18
-9
20
15
1
-4
-6
19
2
10
22
6
-1
4
4
-4
2
-12
-12
-28
1
0
-6
-11
19
-14
10
35
12
-13
11
2
9
-12
-7
-12
-30
9
-6
-2
-20
5
-5
-9
33
-11
-6
28
3
-17
20
15
-2
-9
12
-13
0
0
5
11
-1
-30
-1
19
0
9
1
-8
-10
-3
-10
-24
-11
1
22
-20
21
7
12
20
-5
8
18
-9
16
11
-23
1
-8
12
-7
21
-5
26
9
40
-11
12
16
4
-13
0
6
9
17
24
-3
-5
-10
-20
1
9
2
12
-3
-10
-30
12
-5
-9
16
-5
-21
20
11
-18
15
2
-19
-12
-17
28
-15
-1
25
23
-6
9
-4
-13
20
-1
-6
26
-5
6
17
16
-4
-7
-7
-19
4
-6
-4
8
-13
-2
-19
-9
-7
-36
12
24
8
6
4
0
-13
-34
-25
-22
15
-2
-27
4
8
5
4
0
6
9
-27
-10
-4
-19
0
2
-1
-9
-7
9
4
-1
-45
14
30
30
-2
4
-2
1
17
0
-9
5
9
14
-9
-2
6
9
-17
-10
-4
18
-6
-16
-10
-15
6
-5
0
-1
-8
-1
0
-7
-21
-20
17
-7
-11
-12
-14
5
7
2
-19
-21


In [10]:
#1296
g = [-1, 0, -1, 0, 0, 0, -1, 0, 0, 1, -1, -1, 0, 1, -1, 0, -1, 1, 1, 1, -1, 1, 1, -1, -1, 0, -1, 0, 0, 0, 1, -1, -1, 1, 0, -1, 0, 0, 0, -1, -1, 0, 1, -1, 0, 1, -1, 1, 0, -1, -1, 0, 0, 0, -1, 1, 0, -1, 1, -1, 0, -1, 0, 1, -1, -1, 0, 1, 0, -1, -1, -1, 0, 0, -1, 0, 0, -1, -1, -1, 1, 0, -1, 0, 0, 1, 0, -1, 1, 0, -1, 0, -1, 0, 1, -1, 1, 0, -1, 0, -1, 0, 1, -1, -1, -1, 1, -1, 0, -1, -1, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, -1, 0, -1, -1, 1, 0, 1, 0, 0, -1, 0, 0, -1, 1, 1, 1, 0, 1, -1, 1, 0, 0, 1, 0, -1, 0, -1, -1, -1, -1, 0, -1, 1, 0, -1, 1, 1, 0, -1, -1, 0, 0, 1, 0, -1, 0, 1, -1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 1, 0, 1, -1, 0, -1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, -1, 1, -1, 1, 1, 0, 1, -1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 1, -1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, -1, -1, -1, -1, -1, 0, 0, -1, 0, 0, 0, -1, 1, -1, 0, 0, 0, 0, -1, 1, 1, 0, -1, 1, 1, 0, 1, -1, 1, 0, 0, 1, 0, 0, 0, -1, 0, -1, 0, -1, 0, 0, 1, 0, -1, 0, 0, 1, -1, 0, 0, -1, 0, -1, 0, 1, 1, 0, 0, 0, -1, 0, 1, 0, -1, 0, -1, 0, 0, -1, 1, -1, -1, 1, 1, 0, -1, 0, 1, -1, 0, 1, 1, 0, 0, -1, -1, 0, 0, 1, -1, 0, -1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, -1, -1, 1, 1, 1, 1, 0, 0, 0, 0, -1, 1, 0, 1, 1, -1, 0, 1, -1, 0, -1, -1, -1, -1, 0, 0, 1, 1, 1, 0, 0, 0, -1, 1, 1, 0, 1, 1, -1, 0, 0, -1, 0, 1, 0, -1, 0, -1, -1, -1, 0, 0, -1, 0, 0, 1, 0, 0, -1, -1, -1, -1, 0, 0, -1, -1, 0, 0, -1, 0, 0, 1, -1, 1, 0, 0, 0, 0, 0, -1, -1, 1, 1, 0, 0, 0, -1, 0, 1, 0, 1, -1, -1, -1, 1, 0, 0, 1, 0, -1, 0, 0, 0, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 0, 0, -1, 1, 0, 0, 1, 0, 0, -1, 0, 0, 0, -1, 1, -1, -1, -1, 0, 0, 0, -1, 0, -1, 0, 0, -1, -1, -1, 0, 1, 1, 0, 0, 1, 0, 0, -1, 1, 1, 0, 1, 1, 1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, -1, 0, 0, 0, 0, -1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, -1, 0, 0, 0, -1, 1, 1, 1, -1, 1, 0, 0, 0, 0, -1, -1, -1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, -1, 0, 0, -1, -1, 0, 0, 0, 0, -1, 0, 0, -1, -1, 0, 1, 1, -1, 0, 0, 1, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, -1, 0, 0, 0, -1, -1, 0, 0, 1, 0, -1, 0, 0, -1, 1, 0, 0, 0, 0, -1, 0, 0, -1, -1, 0, 0, 0, 0, -1, 0, 0, 1, 0, 1, -1, -1, 0, 0, 1, 0, 0, 1, 0, 1, -1, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, -1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 1, -1, 0, 0, 0, 0, 1, 0, 0, 1, 1, -1, 0, 0, 1, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 0, 0, 0, 0, -1, 0, 1, 0, -1, 0, 0, -1, 0, -1, 0, -1, -1, -1, 0, 1, -1, 1, -1, 0, 1, -1, 0, 0, -1, 0, -1, 0, 1, 0, 0, -1, -1, 0, 0, 1, -1, 0, -1, 0, 0, 0, 1, -1, 0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 1, 0, -1, -1, 0, 0, 0, 0, 1, -1, -1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, -1, 0, 0, 1, 0, 1, 0, 0, 1, -1, 0, 1, 1, 0, -1, 1, -1, 1, -1, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, -1, -1, 0, 0, 1, 0, 0, -1, -1, 0, -1, -1, 0, 0, 0, -1, 1, 1, 0, -1, 0, 1, -1, 0, 0, -1, -1, 1, 0, 1, -1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 1, -1, 0, 1, 0, 0, -1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, -1, 0, -1, 0, 0, 0, 1, 1, 0, -1, 0, 1, -1, 0, 1, 0, -1, 1, -1, -1, 0, -1, 0, -1, 1, 0, 1, -1, -1, 0, 0, -1, 0, 0, -1, 0, 0, 1, 0, 0, 1, -1, 1, 0, -1, 0, -1, -1, 0, 0, 1, -1, 1, 0, -1, 1, 1, 0, 1, -1, 1, -1, 1, 1, 0, 0, 1, 0, -1, 0, 0, 1, -1, 1, 0, 0, 0, 1, 0, -1, 0, 0, 0, -1, -1, -1, 0, 0, -1, 0, -1, -1, 1, -1, 1, 0, 1, 0, 1, 0, -1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, -1, 0, 0, 0, 0, 1, 1, 0, -1, 0, 0, -1, 0, 0, 0, 1, 1, 0, 0, -1, 0, -1, 1, 0, 0, 0, 1, -1, 1, 1, 1, 0, -1, -1, -1, 0, -1, 0, 1, -1, 1, -1, 1, 0, 0, 1, -1, 1, -1, 1, 1, -1, 0, 0, 0, 1, -1, 1, 0, -1, -1, -1, -1, 0, -1, -1, 0, 0, -1, 0, 0, 1, 1, -1, 0, 1, -1, -1, 1, 0, -1, 0, 0, -1, -1, 1, 0, 0, 0, 1, 1, -1, 0, 0, 0, 0, -1, 1, 0, 1, 0, 0, 0, 1, 0, -1, -1, 0, 1, -1, -1, 1, 1, 0, 1, 0, 0, 1, 0, -1, 0, 1, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 1, 0, -1, 0, -1, -1, 0, 0, -1, 0, 1, -1, 0, 0, -1, 0, 0, 0, 0, 1, 1, -1, 1, 0, 0, -1, -1, -1]

In [3]:
class FFT2:
    def generator(self,n):
        return e^(-2*i*pi / (3*n));
    
    def __init__(self,n):
        self.n = n;

        
        fac = factor(n);
    
        if len(fac) == 2:
            self.Radix2 = fac[0][1];
            self.Radix3 = fac[1][1];

        if len(fac) == 1:
            if fac[0][0] == 2:
                self.Radix2 = fac[0][1];
                self.Radix3 = 0;

            if fac[0][0] == 3:
                self.Radix2 = 0;
                self.Radix3 = fac[0][1];

        self.level = self.Radix2+self.Radix3;

        self.w = self.generator(n);
        self.omega = self.w^n;
    
        tree = zero_matrix(ZZ,self.level+1,n);
        tree[0,0] = 3*n;

        self.zetas = [self.w^0];
                
        #Radix-2 NTT with Cyclotomic Polynomial
        tree[1,0] = tree[0,0] / 6;
        tree[1,1] = 5*tree[0,0] / 6;

        x = self.w^(tree[1,0]);
        self.zetas.append(x);    
    
        #Radix-3 NTT
        for ll in range(1,self.Radix3+1):
            for ii in range(2*3^(ll-1)):
                tree[ll+1,3*ii  ] = tree[ll  ,  ii] / 3;
                tree[ll+1,3*ii+1] = tree[ll  ,  ii] / 3 + tree[0,0]/3;
                tree[ll+1,3*ii+2] = tree[ll  ,  ii] / 3 + 2*tree[0,0]/3;

                x = self.w^(tree[ll+1,3*ii]);
                self.zetas.append(x);
                x = self.w^(tree[ll+1,3*ii]*2);
                self.zetas.append(x);

        #Radix-2 NTT
        for ll in range(self.Radix3+1,self.level):
            for ii in range(2*3^(self.Radix3)*2^(ll-(self.Radix3+1))):
                tree[ll+1,2*ii  ] = tree[ll  ,  ii] / 2;
                tree[ll+1,2*ii+1] = tree[ll  ,  ii] / 2 + tree[0,0]/2;

                x = self.w^(tree[ll+1,2*ii]);
                self.zetas.append(x);

        print(tree)
        
    def fft(self, a):
        k = 1;

        b = a.copy();

        step = self.n/2;
        
        zeta = self.zetas[k];
        k = k+1;

        for ii in range(self.n/2):
            
            t = b[ii + step]*zeta;
            
            b[ii + step] = b[ii] - t + b[ii+step];
            b[ii       ] = b[ii] + t;
            
        step = self.n/6
        while step >= self.n/(6 * 3^(self.Radix3-1)):
            for start in range(0,self.n, 3*step):
                zeta1 = self.zetas[k];
                k = k + 1;
                zeta2 = self.zetas[k];
                k = k + 1;
    
                for ii in range(start,start+step):
                    t1 = zeta1 * b[ii+step];
                    t2 = zeta2 * b[ii+2*step];
                    t3 = self.omega * (t1-t2);
                    
                    b[ii+2*step] = b[ii] - t1 - t3;
                    b[ii+  step] = b[ii] - t2 + t3;
                    b[ii       ] = b[ii] + t1 + t2;
                
            step = step / 3;

        step = 2^(self.Radix2-2)
        while step >= 1:
            for start in range(0,self.n,(step << 1)):
                zeta = self.zetas[k];
                k += 1;

                for ii in range(start,start+step):
                    t = zeta * b[ii+step];

                    b[ii+step] = b[ii]-t;
                    b[ii     ] = b[ii]+t;
            
            step >>= 1;
            
        return b;

In [8]:
n = 768
fft2 = FFT2(n)
#768
g = [-1, 0, -1, 0, 0, 0, -1, 0, 0, 1, -1, -1, 0, 1, -1, 0, -1, 1, 1, 1, -1, 1, 1, -1, -1, 0, -1, 0, 0, 0, 1, -1, -1, 1, 0, -1, 0, 0, 0, -1, -1, 0, 1, -1, 0, 1, -1, 1, 0, -1, -1, 0, 0, 0, -1, 1, 0, -1, 1, -1, 0, -1, 0, 1, -1, -1, 0, 1, 0, -1, -1, -1, 0, 0, -1, 0, 0, -1, -1, -1, 1, 0, -1, 0, 0, 1, 0, -1, 1, 0, -1, 0, -1, 0, 1, -1, 1, 0, -1, 0, -1, 0, 1, -1, -1, -1, 1, -1, 0, -1, -1, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, -1, 0, -1, -1, 1, 0, 1, 0, 0, -1, 0, 0, -1, 1, 1, 1, 0, 1, -1, 1, 0, 0, 1, 0, -1, 0, -1, -1, -1, -1, 0, -1, 1, 0, -1, 1, 1, 0, -1, -1, 0, 0, 1, 0, -1, 0, 1, -1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 1, 0, 1, -1, 0, -1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, -1, 1, -1, 1, 1, 0, 1, -1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 1, -1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, -1, -1, -1, -1, -1, 0, 0, -1, 0, 0, 0, -1, 1, -1, 0, 0, 0, 0, -1, 1, 1, 0, -1, 1, 1, 0, 1, -1, 1, 0, 0, 1, 0, 0, 0, -1, 0, -1, 0, -1, 0, 0, 1, 0, -1, 0, 0, 1, -1, 0, 0, -1, 0, -1, 0, 1, 1, 0, 0, 0, -1, 0, 1, 0, -1, 0, -1, 0, 0, -1, 1, -1, -1, 1, 1, 0, -1, 0, 1, -1, 0, 1, 1, 0, 0, -1, -1, 0, 0, 1, -1, 0, -1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, -1, -1, 1, 1, 1, 1, 0, 0, 0, 0, -1, 1, 0, 1, 1, -1, 0, 1, -1, 0, -1, -1, -1, -1, 0, 0, 1, 1, 1, 0, 0, 0, -1, 1, 1, 0, 1, 1, -1, 0, 0, -1, 0, 1, 0, -1, 0, -1, -1, -1, 0, 0, -1, 0, 0, 1, 0, 0, -1, -1, -1, -1, 0, 0, -1, -1, 0, 0, -1, 0, 0, 1, -1, 1, 0, 0, 0, 0, 0, -1, -1, 1, 1, 0, 0, 0, -1, 0, 1, 0, 1, -1, -1, -1, 1, 0, 0, 1, 0, -1, 0, 0, 0, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 0, 0, -1, 1, 0, 0, 1, 0, 0, -1, 0, 0, 0, -1, 1, -1, -1, -1, 0, 0, 0, -1, 0, -1, 0, 0, -1, -1, -1, 0, 1, 1, 0, 0, 1, 0, 0, -1, 1, 1, 0, 1, 1, 1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, -1, 0, 0, 0, 0, -1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, -1, 0, 0, 0, -1, 1, 1, 1, -1, 1, 0, 0, 0, 0, -1, -1, -1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, -1, 0, 0, -1, -1, 0, 0, 0, 0, -1, 0, 0, -1, -1, 0, 1, 1, -1, 0, 0, 1, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, -1, 0, 0, 0, -1, -1, 0, 0, 1, 0, -1, 0, 0, -1, 1, 0, 0, 0, 0, -1, 0, 0, -1, -1, 0, 0, 0, 0, -1, 0, 0, 1, 0, 1, -1, -1, 0, 0, 1, 0, 0, 1, 0, 1, -1, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, -1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0]
f = [1, -1, 1, 1, -1, 0, -1, 1, 0, 0, 0, 0, 1, 1, 0, 0, -1, 0, 1, 0, 0, 1, -1, 1, -1, 0, 0, 0, 1, -1, -1, 0, 0, 0, -1, 1, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 1, 0, -1, -1, -1, -1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, -1, -1, 1, 0, 1, 0, -1, 1, 0, 0, 1, 0, -1, -1, 1, 1, 0, -1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, -1, 0, 1, 0, -1, 1, 0, 1, 0, 0, -1, -1, 0, 0, -1, 1, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, -1, -1, 1, 1, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, -1, 1, 0, 0, 1, -1, 0, 1, 1, -1, 0, 0, 0, 1, 1, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 1, 1, -1, 0, 0, 0, -1, 0, 1, 1, -1, 1, 1, 0, 0, 0, 0, 1, 0, 0, -1, 1, 0, -1, 0, 0, 0, 0, -1, 0, 1, 1, -1, 1, 0, -1, 1, -1, 0, 0, 1, -1, 1, 0, -1, 0, 0, 1, 0, -1, -1, 1, 1, 0, -1, 0, -1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 1, 0, 0, -1, 0, 1, -1, 1, -1, -1, 1, 1, -1, 0, -1, 1, 0, -1, 1, 1, 0, 0, -1, -1, 1, 1, 0, -1, 0, 1, 0, 1, 0, -1, 0, 1, 1, -1, -1, 1, 0, -1, 0, -1, 0, 1, -1, -1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, -1, -1, 0, 0, 1, 0, 1, -1, 0, 0, 1, 0, 0, 0, -1, 0, 0, 1, 1, -1, 0, 1, 1, -1, 0, 0, -1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 1, -1, -1, 0, -1, 0, 1, 0, 0, 0, -1, -1, -1, 1, -1, 0, 0, -1, 0, 0, 1, 1, 0, -1, 1, 1, 0, 1, 1, 0, -1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, -1, 0, -1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, -1, 0, 1, 0, -1, 1, 1, 0, 1, 0, 0, 1, 1, 0, -1, -1, 0, -1, 1, 0, 0, 1, 1, 0, 1, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, -1, -1, 0, 0, 0, 0, -1, -1, -1, 0, 1, 0, -1, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, -1, 0, 1, -1, 0, -1, 0, 1, 0, 1, 0, 1, -1, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, -1, 0, 1, 1, 1, 0, 0, 0, 0, -1, -1, 1, -1, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, -1, 1, 1, 0, 1, 0, 0, 0, -1, 0, 0, 1, 1, 0, 0, 1, 0, -1, -1, 1, 1, 0, 1, 0, -1, -1, 0, 1, 1, 1, 0, -1, 0, 1, 0, 0, 0, 0, 0, 1, -1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, -1, 1, -1, 0, -1, 0, 0, 1, 1, -1, 0, 1, 0, 0, 0, 0, 1, -1, -1, 0, 1, 0, 0, -1, 0, -1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, -1, -1, -1, 0, 1, -1, 1, 1, -1, 0, 0, 0, 0, -1, 1, 0, 1, 0, 0, 1, -1, 0, 0, 1, -1, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, -1, 0, 1, -1, 0, 0, 0, 0, 0, 1, -1, 0, 1, 0, 0, 0, 1, -1, -1, 0, 0, -1, 0, 1, 1, -1, -1, 0, 1, 0, -1, -1, -1, 0, 0, 0, -1, 1, 1, -1, -1, 0, -1, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 1, -1, -1, 0, 0, 0, 1, 0, 1, 0, 1, -1, -1, 0, 0, 1, 0, 0, -1, -1, -1, 1, 1, 0, 1, 1, 0, -1, -1, 1, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 1, -1, -1, 0, -1, 1, 0, 0, 0, 1, 1]

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
[2304    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

In [9]:
ghat = fft2.fft(g)

In [ ]:
for i in range(n):
    print(round(ghat[i].real()))

-24
-11
-13
16
-4
-4
-18
-18
-12
-18
-3
-11
-11
30
-12
23
-2
-2
10
-1
-8
8
-3
-4
14
24
-14
-1
-24
1
-10
17
1
6
3
18
-4
17
-3
18
-2
-6
6
-3
-1
-18
-15
-13
-7
1
3
-17
-5
-2
-19
13
-1
-15
10
10
13
-22
-22
2
12
21
2
-5
6
-29
21
-15
1
-7
1
22
-2
-6
-8
-7
4
-1
3
19
8
17
6
5
-16
10
-17
-3
22
18
-9
20
15
1
-4
-6
19
2
10
22
6
-1
4
4
-4
2
-12
-12
-28
1
0
-6
-11
19
-14
10
35
12
-13
11
2
9
-12
-7
-12
-30
9
-6
-2
-20
5
-5
-9
33
-11
-6
28
3
-17
20
15
-2
-9
12
-13
0
0
5
11
-1
-30
-1
19
0
9
1
-8
-10
-3
-10
-24
-11
1
22
-20
